In [46]:
using BenchmarkTools
using LoopVectorization

In [78]:
W = zeros(256,10);
y_hat = 10
learning_rate = 0.000001
x = rand(256);
x_view = view(x,1:length(x));

In [100]:
function Wj_eq_Wj_plus_lr_x_vectorized!(W, j, x, learning_rate)
   @turbo W[:, j] .= view(W,:, j) .- learning_rate .* x;
end

Wj_eq_Wj_plus_lr_x_vectorized! (generic function with 1 method)

In [101]:
@btime Wj_eq_Wj_plus_lr_x_vectorized!($W, $y_hat, $x, $learning_rate);
@btime Wj_eq_Wj_plus_lr_x_vectorized!($W, $y_hat, $x_view, $learning_rate);

  56.021 ns (0 allocations: 0 bytes)
  56.318 ns (0 allocations: 0 bytes)


In [98]:

function Wj_eq_Wj_plus_lr_x!(W, j, x, learning_rate)
    #h.W[:, j ]  .= view(h.W,:, j )  .+ learning_rate .* x
    @turbo for i in eachindex(axes(W,1))     
        W[i, j] = W[i,j] + learning_rate * x[i]
    end
end

Wj_eq_Wj_plus_lr_x! (generic function with 1 method)

In [99]:
@btime Wj_eq_Wj_plus_lr_x!($W, $y_hat, $x, $learning_rate)
@btime Wj_eq_Wj_plus_lr_x!($W, $y_hat, $x_view, $learning_rate)

  31.648 ns (0 allocations: 0 bytes)
  32.612 ns (0 allocations: 0 bytes)


In [43]:

function update_W_col_j_with_x!(W, j, x, learning_rate)
    #h.W[:, j ]  .= view(h.W,:, j )  .+ learning_rate .* x
    W[:,j] .= muladd(x, learning_rate, x)
end

update_W_col_j_with_x! (generic function with 1 method)

In [45]:
@btime update_W_col_j_with_x!(W, y_hat, x, learning_rate);

  237.033 ns (3 allocations: 4.30 KiB)
